In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn.modules.distance import PairwiseDistance
from datasets.AIHubDataset import AIHubDataset
from validate_aihub import validate_aihub
from tqdm import tqdm
from model.inceptionresnetv2 import InceptionResnetV2Triplet
from model.mobilenetv2 import MobileNetV2Triplet
from model.resnet import (
    Resnet18Triplet,
    Resnet34Triplet,
    Resnet50Triplet,
    Resnet101Triplet,
    Resnet152Triplet,
)

In [ ]:
from insightface.recognition.arcface_torch.backbones import get_model

In [ ]:
# dataroot = "data/face-image/train_aihub_family"
# lfw_dataroot = "data/face-image/lfw_224"
aihub_dataroot = "data/face-image/test_aihub_family"
# training_dataset_csv_path = "aihub_train.csv"
epochs = 2  # 150
# iterations_per_epoch = 10  # 5000
model_architecture = "r50"
pretrained = False # False
# embedding_dimension = 512
# # num_human_identities_per_batch = 32
# # batch_size = 100  # 544
lfw_batch_size = 200
# # resume_path = ""
num_workers = 4
# optimizer = "adagrad"
# learning_rate = 0.075
# margin = 0.2
image_size = 112
# use_semihard_negatives = False
# training_triplets_path = None

In [ ]:
# Define image data pre-processing transforms
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
aihub_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
TASK_CATEGORY = "Individuals"
aihub_dataloader = torch.utils.data.DataLoader(
    dataset=AIHubDataset(
        dir=aihub_dataroot,
        pairs_path=f"data/pairs/test/pairs_{TASK_CATEGORY}.txt",
        transform=aihub_transforms,
    ),
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)

In [ ]:
model = get_model(model_architecture, fp16=False)
model_weights = torch.load("/home/jupyter/face/utils/model/arcface/backbone.pth")
model.load_state_dict(model_weights)
model.cuda().eval()

In [ ]:
# Evaluation pass on LFW dataset
print(f"Task category")
print(f"{TASK_CATEGORY}")
for epoch in range(epochs):
    best_distances = validate_aihub(
        model=model,
        aihub_dataloader=aihub_dataloader,
        model_architecture=model_architecture,
        epoch=epoch
    )
    break

In [ ]:
from torch.nn.modules.distance import PairwiseDistance


model.eval()
with torch.no_grad():
    l2_distance = PairwiseDistance(p=2)
    distances, labels = [], []

    print("Validating on AIHUB! ...")
    progress_bar = enumerate(tqdm(aihub_dataloader))

    for batch_index, (data_a, data_b, label) in progress_bar:
        data_a = data_a.cuda()
        data_b = data_b.cuda()

        output_a, output_b = model(data_a), model(data_b)
        distance = l2_distance.forward(output_a, output_b)  # Euclidean distance

        distances.append(distance.cpu().detach().numpy())
        labels.append(label.cpu().detach().numpy())
        break